In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

data_sets = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(filename)
        if filename == "test_essays.csv":
            test = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays.csv":
            train = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays_RDizzl3_seven_v2.csv":
            generated_text = pd.read_csv(os.path.join(dirname, filename))
        data_sets.append(pd.read_csv(os.path.join(dirname, filename)))
    print(f"{len(data_sets)} data sets loaded in.")
    
    
#test = train[10:10000]
#sample_sub, training_prompts, test, train, _, _, _, generated_text = data_sets

0 data sets loaded in.
sample_submission.csv
train_prompts.csv
test_essays.csv
train_essays.csv
4 data sets loaded in.
train_essays_RDizzl3_seven_v2.csv
train_essays_7_prompts_v2.csv
train_essays_7_prompts.csv
train_essays_RDizzl3_seven_v1.csv
8 data sets loaded in.


In [2]:
print("Supplementary dataset features:")
print(generated_text.dtypes)
print("\nOriginal dataset features:")
print(train.dtypes)


Supplementary dataset features:
text     object
label     int64
dtype: object

Original dataset features:
id           object
prompt_id     int64
text         object
generated     int64
dtype: object


As we can see, we need to make it so that these share a universal format so that we can combine the data together.

In [3]:
test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [4]:
# Rename the label column to match original dataset
generated_text.rename(columns={'label': 'generated'}, inplace=True)
generated_text

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
17246,"Dear Senator,\n\nI am writing to you today to ...",1
17247,"Dear Senator,\n\nI am writing to you today to ...",1
17248,"Dear Senator,\n\nI am writing to you today to ...",1
17249,"Dear Senator,\n\nI am writing to you today to ...",1


In [5]:
# Drop the two columns we don't need for training
train.drop('id', axis='columns', inplace=True)
train.drop('prompt_id', axis='columns', inplace=True)
train

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [6]:
training_set = pd.concat([generated_text, train])

In [7]:
training_set.describe()

,generated
count,18629.000000
mean,0.161415
std,0.367923
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import keras_nlp
import keras_core as keras



import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
   

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping

Using TensorFlow backend


In [9]:
isRealTest = True

if isRealTest:
    X_train = training_set['text']
    y_train = training_set['generated']
    
    X_test = test['text']
    X_testIDs = test['id']
    
    
else:
    X = training_set['text']
    y = training_set['generated']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

vectorizer = TfidfVectorizer(max_features = 10000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.fit_transform(X_test).toarray()

desired_shape = 10000
pad_width = desired_shape - X_test_tfidf.shape[1]
# X_train_tfidf_padded = np.pad(X_train_tfidf, ((0, 0), (0, pad_width)), mode='constant')
X_test_tfidf_padded = np.pad(X_test_tfidf, ((0, 0), (0, pad_width)), mode='constant')

In [10]:
# max_words = 10000
# padding_length = 200

# tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
# tokenizer.fit_on_texts(X_train)

# # Tokenize them into vectors
# # Each unique word represented by an integer
# X_train_sequences = tokenizer.texts_to_sequences(X_train)
# X_test_sequences = tokenizer.texts_to_sequences(X_test)

# # Pad our data to ensure consistent length
# X_train_padded = pad_sequences(X_train_sequences, maxlen=padding_length, padding='post', truncating='post')
# X_test_padded = pad_sequences(X_test_sequences, maxlen=padding_length, padding='post', truncating='post')

# vectorizer = TfidfVectorizer(max_features = 10000)
# X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
# X_test_tfidf = vectorizer.fit_transform(X_test).toarray()

In [11]:
print(X_test_tfidf.shape)
print(X_test_tfidf_padded.shape)
# print(X_test_tfidf)

(3, 5)
(3, 10000)


In [12]:
## Our first model, the LSTM
LSTM_model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(64),
    Dense(1, activation='sigmoid')
])


## For a TF-IDF model
TFIDF_model = Sequential([
    Dense(units = 128, activation = 'relu'),
    Dense(1, activation='sigmoid')
])

# Creating the model
TFIDF_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# TFIDF_model.build()
# TFIDF_model.summary()

In [13]:
X_train_tfidf.shape[1]

10000

In [14]:
# Train the model
TFIDF_model.fit(X_train_tfidf, y_train, epochs=5, batch_size=32)

# y_train = y_train.astype(np.float32)
# if not isRealTest:
#     y_test  = y_test .astype(np.float32)

# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



Epoch 1/5
583/583 [==============================] - 4s 6ms/step - loss: 0.0667 - accuracy: 0.9791
Epoch 2/5
583/583 [==============================] - 3s 6ms/step - loss: 0.0029 - accuracy: 0.9996
Epoch 3/5
583/583 [==============================] - 4s 6ms/step - loss: 8.1331e-04 - accuracy: 1.0000
Epoch 4/5
583/583 [==============================] - 3s 6ms/step - loss: 3.4238e-04 - accuracy: 1.0000
Epoch 5/5
583/583 [==============================] - 3s 6ms/step - loss: 1.8371e-04 - accuracy: 1.0000


In [15]:
if not isRealTest:
    # Evaluate the model on the test set
    loss, accuracy = TFIDF_model.evaluate(X_test_tfidf_padded, y_test)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [16]:
print(X_test_tfidf.shape)

(3, 5)


In [17]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on the test set
y_pred = TFIDF_model.predict(X_test_tfidf_padded)

# Convert probabilities to class labels
y_pred_labels = np.round(y_pred)

1/1 [==============================] - 0s 64ms/step


In [18]:
if not isRealTest:
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification report
    class_report = classification_report(y_test, y_pred_labels)
    print("\nClassification Report:")
    print(class_report)

if isRealTest:
    submission = pd.DataFrame({'id': list(X_testIDs), 'generated': y_pred.flatten()})

In [19]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [20]:
submission

,id,generated
0,0000aaaa,0.247398
1,1111bbbb,0.407894
2,2222cccc,0.232640
